In [15]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib


In [16]:
# Load Datasets
crop_data = pd.read_csv("dataset/Crop_recommendation.csv")
fertilizer_data = pd.read_csv("dataset/Fertilizer Prediction.csv")


In [17]:
# Encode categorical features in fertilizer dataset
fertilizer_label_encoder = LabelEncoder()
fertilizer_data['Soil Type'] = fertilizer_data['Soil Type'].astype('category').cat.codes
fertilizer_data['Crop Type'] = fertilizer_data['Crop Type'].astype('category').cat.codes
fertilizer_data['Fertilizer Name'] = fertilizer_label_encoder.fit_transform(fertilizer_data['Fertilizer Name'])


In [18]:
# Encode crop labels in crop dataset
crop_label_encoder = LabelEncoder()
crop_data['label'] = crop_label_encoder.fit_transform(crop_data['label'])


In [19]:
# Split features and target for Crop Prediction
X_crop = crop_data.drop('label', axis=1)
y_crop = crop_data['label']
X_train_crop, X_test_crop, y_train_crop, y_test_crop = train_test_split(X_crop, y_crop, test_size=0.2, random_state=42)


In [20]:
# Train Decision Tree for Crop Prediction
crop_model = DecisionTreeClassifier()
crop_model.fit(X_train_crop, y_train_crop)


DecisionTreeClassifier()

In [21]:
# Evaluate Crop Model
crop_pred = crop_model.predict(X_test_crop)
print("Crop Model Accuracy:", accuracy_score(y_test_crop, crop_pred))
print(classification_report(y_test_crop, crop_pred))


Crop Model Accuracy: 0.9818181818181818
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        21
           2       0.95      1.00      0.98        20
           3       1.00      1.00      1.00        26
           4       0.96      1.00      0.98        27
           5       1.00      1.00      1.00        17
           6       1.00      1.00      1.00        17
           7       1.00      1.00      1.00        14
           8       0.91      0.87      0.89        23
           9       1.00      1.00      1.00        20
          10       0.92      1.00      0.96        11
          11       1.00      0.95      0.98        21
          12       1.00      1.00      1.00        19
          13       1.00      0.92      0.96        24
          14       1.00      1.00      1.00        19
          15       1.00      1.00      1.00        17
          16       1.00      1.00      1.

In [22]:
# Split features and target for Fertilizer Prediction
X_fertilizer = fertilizer_data.drop('Fertilizer Name', axis=1)
y_fertilizer = fertilizer_data['Fertilizer Name']
X_train_fert, X_test_fert, y_train_fert, y_test_fert = train_test_split(X_fertilizer, y_fertilizer, test_size=0.2, random_state=42)


In [23]:
# Train Decision Tree for Fertilizer Prediction
fertilizer_model = DecisionTreeClassifier()
fertilizer_model.fit(X_train_fert, y_train_fert)


DecisionTreeClassifier()

In [24]:
# Evaluate Fertilizer Model
fertilizer_pred = fertilizer_model.predict(X_test_fert)
print("Fertilizer Model Accuracy:", accuracy_score(y_test_fert, fertilizer_pred))
print(classification_report(y_test_fert, fertilizer_pred))


Fertilizer Model Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         6

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [25]:
# Save models
joblib.dump(crop_model, "crop_model.pkl")
joblib.dump(fertilizer_model, "fertilizer_model.pkl")
joblib.dump(StandardScaler().fit(X_crop), "scaler.pkl")


['scaler.pkl']

In [26]:
# Load models
scaler = joblib.load("scaler.pkl")
crop_model = joblib.load("crop_model.pkl")
fertilizer_model = joblib.load("fertilizer_model.pkl")


In [27]:
# Prediction Function
def predict_crop_and_fertilizer(N, P, K, temperature, humidity, ph, rainfall, soil_type, crop_type, moisture):
    crop_input = pd.DataFrame([[N, P, K, temperature, humidity, ph, rainfall]], columns=X_crop.columns)
    crop_pred = crop_model.predict(crop_input)[0]
    predicted_crop = crop_label_encoder.inverse_transform([crop_pred])[0]
    
    fertilizer_input = pd.DataFrame([[temperature, humidity, moisture, soil_type, crop_type, N, P, K]], columns=X_fertilizer.columns)
    fertilizer_pred = fertilizer_model.predict(fertilizer_input)[0]
    predicted_fertilizer = fertilizer_label_encoder.inverse_transform([fertilizer_pred])[0]
    
    return f"Recommended Crop: {predicted_crop}, Recommended Fertilizer: {predicted_fertilizer}"


In [ ]:
# Example Prediction
print(predict_crop_and_fertilizer(90, 42, 43, 20, 82, 6.5, 200, 2, 3, 45))

Recommended Crop: rice, Recommended Fertilizer: DAP


In [30]:
print(predict_crop_and_fertilizer(50, 30, 20, 25, 75, 6.8, 150, 1, 2, 40))   

Recommended Crop: orange, Recommended Fertilizer: 28-28


In [31]:
print(predict_crop_and_fertilizer(80, 40, 35, 22, 80, 7.0, 180, 3, 1, 50))  

Recommended Crop: rice, Recommended Fertilizer: DAP


In [ ]:
print(predict_crop_and_fertilizer(60, 25, 30, 18, 70, 6.2, 120, 1, 4, 35))  

Recommended Crop: rice, Recommended Fertilizer: DAP


In [33]:
print(predict_crop_and_fertilizer(95, 50, 40, 30, 85, 7.1, 220, 2, 5, 55)) 

Recommended Crop: rice, Recommended Fertilizer: DAP
